In [52]:
import pandas as pd
import numpy as np
from pathlib import Path

In [53]:
df = pd.read_csv('../data/raw/f1_dataset_base.csv')
df

,year,round,date,circuit_name,driver_code,driver_number,team_name,grid_position,classified_position,status,points_real
0,2000,1,2000-03-12 00:00:00+00:00,Australian Grand Prix,MSC,3,Ferrari,3.0,1,Finished,10.0
1,2000,1,2000-03-12 00:00:00+00:00,Australian Grand Prix,BAR,4,Ferrari,4.0,2,Finished,6.0
2,2000,1,2000-03-12 00:00:00+00:00,Australian Grand Prix,SCH,9,Williams,11.0,3,Finished,4.0
3,2000,1,2000-03-12 00:00:00+00:00,Australian Grand Prix,VIL,22,BAR,8.0,4,Finished,3.0
4,2000,1,2000-03-12 00:00:00+00:00,Australian Grand Prix,FIS,11,Benetton,9.0,5,Finished,2.0
...,...,...,...,...,...,...,...,...,...,...,...
9980,2025,22,2025-11-23 04:00:00,Las Vegas Grand Prix,NOR,4,McLaren,1.0,D,Disqualified,0.0
9981,2025,22,2025-11-23 04:00:00,Las Vegas Grand Prix,PIA,81,McLaren,5.0,D,Disqualified,0.0
9982,2025,22,2025-11-23 04:00:00,Las Vegas Grand Prix,ALB,23,Williams,16.0,R,Retired,0.0
9983,2025,22,2025-11-23 04:00:00,Las Vegas Grand Prix,BOR,5,Kick Sauber,18.0,R,Retired,0.0


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9985 entries, 0 to 9984
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   year                 9985 non-null   int64  
 1   round                9985 non-null   int64  
 2   date                 9985 non-null   object 
 3   circuit_name         9985 non-null   object 
 4   driver_code          9900 non-null   object 
 5   driver_number        9985 non-null   int64  
 6   team_name            9985 non-null   object 
 7   grid_position        9982 non-null   float64
 8   classified_position  9985 non-null   object 
 9   status               9985 non-null   object 
 10  points_real          9985 non-null   float64
dtypes: float64(2), int64(3), object(6)
memory usage: 858.2+ KB


In [55]:
# Hay que cambiar classified_position de object a numérico

In [56]:
df.describe()

,year,round,driver_number,grid_position,points_real
count,9985.000000,9985.000000,9985.000000,9982.000000,9985.000000
mean,2013.588883,10.292539,19.373961,10.701262,3.931597
std,7.146401,5.762967,19.571444,6.131210,6.296007
min,2000.000000,1.000000,1.000000,0.000000,0.000000
25%,2008.000000,5.000000,7.000000,5.000000,0.000000
50%,2014.000000,10.000000,14.000000,11.000000,0.000000
75%,2020.000000,15.000000,22.000000,16.000000,6.000000
max,2025.000000,24.000000,99.000000,24.000000,50.000000


In [57]:
df.groupby('year')['driver_code'].nunique().sort_index()

year
2000    12
2001    14
2002    16
2003    15
2004    19
2005    27
2006    27
2007    26
2008    22
2009    25
2010    27
2011    28
2012    25
2013    23
2014    24
2015    22
2016    24
2017    25
2018    20
2019    20
2020    23
2021    21
2022    22
2023    22
2024    24
2025    21
Name: driver_code, dtype: int64

In [58]:
# Hasta el año 2000 no aparecen todos los pilotos que participan en el campeonato
# Por lo tanto estos años no pueden aparecer como referencia para entrenar al modelo ya que estan incompletos

In [59]:
df['points_real'].nunique()

30

In [60]:
df['points_real'].value_counts()

points_real
0.0     5452
4.0      497
2.0      495
6.0      491
10.0     490
1.0      479
8.0      442
12.0     315
15.0     313
18.0     302
25.0     288
3.0      175
5.0      128
26.0      37
19.0      23
16.0      13
13.0      11
11.0       8
7.0        8
9.0        7
0.5        2
2.5        1
1.5        1
20.0       1
30.0       1
36.0       1
50.0       1
24.0       1
12.5       1
7.5        1
Name: count, dtype: int64

In [61]:
# El sistema de puntuación ha ido variando con los años, por lo que para que un resultado sea igual en 2005 que en 2025
# tenemos que estandarizar los puntos, los cuales van a ser nuestro indicador para ordenar la clasificación

In [62]:
df['status'].nunique()

97

In [63]:
df['status'].value_counts()

status
Finished          4967
+1 Lap            1982
+2 Laps            447
Collision          362
Lapped             290
                  ... 
Excluded             1
Water pump           1
Cooling system       1
Debris               1
Fuel leak            1
Name: count, Length: 97, dtype: int64

In [64]:
df['classified_position'].nunique()

27

In [65]:
df['classified_position'].value_counts()

classified_position
R     1818
2      501
1      501
3      501
4      500
5      496
6      486
7      481
8      479
9      473
10     466
11     452
12     449
13     429
14     404
15     379
16     334
17     284
18     203
19     128
20      70
W       65
D       34
21      30
22      16
23       5
24       1
Name: count, dtype: int64

In [66]:
df[df['classified_position'] == 'R']

,year,round,date,circuit_name,driver_code,driver_number,team_name,grid_position,classified_position,status,points_real
9,2000,1,2000-03-12 00:00:00+00:00,Australian Grand Prix,BUT,10,Williams,21.0,R,Engine,0.0
10,2000,1,2000-03-12 00:00:00+00:00,Australian Grand Prix,TRU,6,Jordan,6.0,R,Engine,0.0
11,2000,1,2000-03-12 00:00:00+00:00,Australian Grand Prix,COU,2,McLaren,2.0,R,Engine,0.0
12,2000,1,2000-03-12 00:00:00+00:00,Australian Grand Prix,DLR,18,Arrows,12.0,R,Suspension,0.0
22,2000,2,2000-03-26 00:00:00+00:00,Brazilian Grand Prix,BAR,4,Ferrari,4.0,R,Hydraulics,0.0
...,...,...,...,...,...,...,...,...,...,...,...
9963,2025,21,2025-11-09 17:00:00,São Paulo Grand Prix,LEC,16,Ferrari,3.0,R,Retired,0.0
9964,2025,21,2025-11-09 17:00:00,São Paulo Grand Prix,BOR,5,Kick Sauber,18.0,R,Retired,0.0
9982,2025,22,2025-11-23 04:00:00,Las Vegas Grand Prix,ALB,23,Williams,16.0,R,Retired,0.0
9983,2025,22,2025-11-23 04:00:00,Las Vegas Grand Prix,BOR,5,Kick Sauber,18.0,R,Retired,0.0


In [67]:
df[df['classified_position'] == 'D']

,year,round,date,circuit_name,driver_code,driver_number,team_name,grid_position,classified_position,status,points_real
21,2000,2,2000-03-26 00:00:00+00:00,Brazilian Grand Prix,COU,2,McLaren,2.0,D,Disqualified,0.0
293,2001,6,2001-05-13 00:00:00+00:00,Austrian Grand Prix,TRU,12,Jordan,5.0,D,Disqualified,0.0
1085,2004,8,2004-06-13 00:00:00+00:00,Canadian Grand Prix,SCH,4,Williams,1.0,D,Disqualified,0.0
1086,2004,8,2004-06-13 00:00:00+00:00,Canadian Grand Prix,MON,3,Williams,4.0,D,Disqualified,0.0
1100,2004,9,2004-06-20 00:00:00+00:00,United States Grand Prix,MON,3,Williams,5.0,D,Disqualified,0.0
1333,2005,4,2005-04-24 14:00:00+00:00,San Marino Grand Prix,BUT,3,BAR,3.0,D,Disqualified,0.0
1334,2005,4,2005-04-24 14:00:00+00:00,San Marino Grand Prix,SAT,4,BAR,6.0,D,Disqualified,0.0
1407,2005,8,2005-06-12 13:00:00+00:00,Canadian Grand Prix,MON,10,McLaren,5.0,D,Disqualified,0.0
1612,2005,18,2005-10-09 14:00:00+00:00,Japanese Grand Prix,SAT,4,BAR,5.0,D,Disqualified,0.0
1890,2006,12,2006-07-30 14:00:00+00:00,German Grand Prix,ALB,19,MF1,21.0,D,Disqualified,0.0


In [68]:
# Hay que marcar como abandono a todos aquellos que no han terminado la carrera o han sido descalificados por el motivo que sea ya que no se les pueden asignar puntos ya que no tienen posicion final
# y por lo tanto hay que tenerlo en cuenta a la hora de crear las features en un futuro, de otro modo podria dar error si incluimos estas carreras en sus estadisticas

In [70]:
INPUT_FILE = '../data/raw/f1_dataset_base.csv'
OUTPUT_FILE = '../data/processed/f1_clean.csv'

def calcular_puntos_modernos(position):
    puntos_map = {
        1: 25, 2: 18, 3: 15, 4: 12, 5: 10,
        6: 8, 7: 6, 8: 4, 9: 2, 10: 1
    }
    return puntos_map.get(position, 0)

def limpiar_dataset(df):
    df = df.copy()
    
    # 1. FILTRAR DESDE 2005
    df = df[df['year'] >= 2005].copy()
    
    # 2. CONVERTIR POSICIONES A NUMÉRICO
    df['position'] = pd.to_numeric(df['classified_position'], errors='coerce')
    df['position'] = df['position'].fillna(999).astype(int)
    
    # 3. RENOMBRAR COLUMNAS
    df = df.rename(columns={
        'driver_code': 'driver',
        'team_name': 'team'
    })
    
    # 4. RECALCULAR PUNTOS CON SISTEMA MODERNO
    df['puntos_carrera'] = df['position'].apply(calcular_puntos_modernos)
    
    # 5. MARCAR ABANDONOS
    df['es_abandono'] = df['position'] == 999
    
    # 6. SELECCIONAR COLUMNAS FINALES
    columnas_finales = [
        'year', 'round', 'driver', 'team', 
        'position', 'puntos_carrera', 'es_abandono'
    ]
    df = df[columnas_finales]
    
    # 7. ORDENAR
    df = df.sort_values(['year', 'round', 'position']).reset_index(drop=True)
    
    return df


if __name__ == "__main__":
    
    df_raw = pd.read_csv(INPUT_FILE)

    df_clean = limpiar_dataset(df_raw)
    
    Path(OUTPUT_FILE).parent.mkdir(parents=True, exist_ok=True)
    df_clean.to_csv(OUTPUT_FILE, index=False)


In [71]:
df_clean

,year,round,driver,team,position,puntos_carrera,es_abandono
0,2005,1,FIS,Renault,1,25,False
1,2005,1,BAR,Ferrari,2,18,False
2,2005,1,ALO,Renault,3,15,False
3,2005,1,COU,Red Bull,4,12,False
4,2005,1,WEB,Williams,5,10,False
...,...,...,...,...,...,...,...
8720,2025,22,NOR,McLaren,999,0,True
8721,2025,22,PIA,McLaren,999,0,True
8722,2025,22,ALB,Williams,999,0,True
8723,2025,22,BOR,Kick Sauber,999,0,True


In [ ]:
# Ya esta el dataset listo para calcular el resto de features dinámicas